# Writing a Pandas Dataframe to the database

#### Setup environemnt & connection

In [3]:
!pip install pyexasol
!pip install websocket-client ujson
!pip install pyodbc

In [ ]:
# imports
import pandas as pd
import pyexasol

host = '192.168.56.101/E176DDD5013BFA9B9FAF6E470EF099181698FC0ED1817158CE84911F02345B5E:8563'  # Don't forget to change the fingerprint
user = 'sys'
password = 'exasol'

# Connect to Exasol
conn = pyexasol.connect(dsn=host, 
                        user=user, 
                        password=password, 
                        debug=False, 
                        protocol_version=pyexasol.PROTOCOL_V1)


#### Create Pandas Dataframe (Or read in our data here)

In [25]:
# Here is some made up data - trying integer, float, and string data

data = {
    'TestString' : ['hi'] * 500,
    'TestInt' : [4] * 250 + [3] * 250,
    'TestFloat' : [3.0] * 500
}
df = pd.DataFrame(data)

# Verify it was created
print(df.shape, "\n", df.head)

(500, 3) 
 <bound method NDFrame.head of     TestString  TestInt  TestFloat
0           hi        4        3.0
1           hi        4        3.0
2           hi        4        3.0
3           hi        4        3.0
4           hi        4        3.0
..         ...      ...        ...
495         hi        3        3.0
496         hi        3        3.0
497         hi        3        3.0
498         hi        3        3.0
499         hi        3        3.0

[500 rows x 3 columns]>


#### Create our table on the database - Run this one time

In [33]:
create_table_query = '''
CREATE TABLE AOL_SCHEMA.TEST_TABLE1 (
    TestString VARCHAR(100),
    TestInt INTEGER,
    TestFloat DECIMAL(10,0)
)
'''

conn.execute(create_table_query)

<ExaStatement session_id=1814528615647245490 stmt_idx=24>

#### Write your data to the table - Running this multiple times will append more data

In [19]:
conn.import_from_pandas(df, table=('AOL_SCHEMA', 'TEST_TABLE1'))

#### Test that it was actually written

In [20]:
# Query from the database

# Abstract reading and printing
def print_query(query, connection):
    stmt = connection.export_to_pandas(query)
    print(stmt.head())
    return stmt

q = '''SELECT * FROM AOL_SCHEMA.TEST_TABLE1'''

print("It works! Note that multiple writes simply appends to the table")
print_query(q, conn)

It works! Note that multiple writes simply appends to the table
  TESTSTRING  TESTINT  TESTFLOAT
0         hi        3          3
1         hi        3          3
2         hi        3          3
3         hi        3          3
4         hi        3          3


,TESTSTRING,TESTINT,TESTFLOAT
0,hi,3,3
1,hi,3,3
2,hi,3,3
3,hi,3,3
4,hi,3,3
...,...,...,...
1495,hi,4,3
1496,hi,4,3
1497,hi,4,3
1498,hi,4,3


#### Now remove data from the table

In [ ]:
delete_query = '''
DELETE FROM AOL_SCHEMA.TEST_TABLE1
WHERE AOL_SCHEMA.TEST_TABLE1.TESTINT = 4
'''

# Delete All
#delete_query = "DELETE FROM AOL_SCHEMA.TEST_TABLE1 "

conn.execute(delete_query)

<ExaStatement session_id=1814528615647245490 stmt_idx=15>

In [24]:
# Verify that it works
q = '''SELECT * FROM AOL_SCHEMA.TEST_TABLE1'''
print_query(q, conn)

Empty DataFrame
Columns: [TESTSTRING, TESTINT, TESTFLOAT]
Index: []


,TESTSTRING,TESTINT,TESTFLOAT


#### Delete the table in SQL - Drop Table

In [37]:
drop_table_query = "DROP TABLE AOL_SCHEMA.TEST_TABLE1"
conn.execute(drop_table_query)

<ExaStatement session_id=1814528615647245490 stmt_idx=28>

In [38]:
# Verify that it works
q = '''SELECT * FROM AOL_SCHEMA.TEST_TABLE1'''

try:
    conn.export_to_pandas(q)
    print("Failure, table exists")
except:
    print("Success, table doesn't exist")

Success, table doesn't exist
